In [ ]:
import cv2
import os
from PIL import Image
from cv2 import resize
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [ ]:
image_directory = 'dataset/'

In [ ]:
Mild_heart_images = os.listdir(image_directory + 'Mild/')
Moderate_heart_images = os.listdir(image_directory + 'Moderate/')
No_heart_images = os.listdir(image_directory + 'No/')
Proliferate_heart_images = os.listdir(image_directory + 'Proliferate/')
Severe_heart_images = os.listdir(image_directory + 'Severe/')

In [ ]:
dataset = []
label = []

In [ ]:
INPUT_SIZE = 244

In [ ]:
for i , image_name in enumerate(No_heart_images):
    if(image_name.split('.')[1] == 'png'):
        image = cv2.imread(image_directory + 'No/'+ image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(0)

In [ ]:
for i , image_name in enumerate(Mild_heart_images):
    if(image_name.split('.')[1] == 'png'):
        image = cv2.imread(image_directory + 'Mild/'+ image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(1)

In [ ]:
for i , image_name in enumerate(Moderate_heart_images):
    if(image_name.split('.')[1] == 'png'):
        image = cv2.imread(image_directory + 'Moderate/'+ image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(2)

In [ ]:
for i , image_name in enumerate(Severe_heart_images):
    if(image_name.split('.')[1] == 'png'):
        image = cv2.imread(image_directory + 'Severe/'+ image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(3)

In [ ]:
for i , image_name in enumerate(Proliferate_heart_images):
    if(image_name.split('.')[1] == 'png'):
        image = cv2.imread(image_directory + 'Proliferate/'+ image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(4)

In [ ]:
print(len(dataset))
print(len(label))

In [ ]:
dataset=np.array(dataset)
label=np.array(label)

In [ ]:
train_images, test_images, train_labels, test_labels = train_test_split(dataset, label, test_size=0.2, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
    layers.experimental.preprocessing.RandomTranslation(0.1, 0.1),
    layers.experimental.preprocessing.RandomFlip(mode="horizontal"),
])

In [ ]:
model = models.Sequential([
    layers.Input(shape=(244, 244, 3)),
    data_augmentation,
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(5, activation='softmax')
])

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=0.00001)

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [ ]:
model.fit(data_augmentation(train_images), train_labels,
                    epochs=20,
                    validation_data=(data_augmentation(val_images), val_labels),
                    callbacks=[reduce_lr, early_stop])

In [ ]:
test_loss, test_acc = model.evaluate(data_augmentation(test_images), test_labels)

In [ ]:
print('Test accuracy:', test_acc*100 )

In [ ]:
model.save('heart_attack_prediction_model.h5')

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split

# Function to load and preprocess images
def load_and_preprocess_image(image_path, input_size):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (input_size, input_size))
    image = image.astype('float32') / 255.0
    return image

# Load images and labels
def load_data(image_dir):
    dataset = []
    labels = []
    label_map = {'No': 0, 'Mild': 1, 'Moderate': 2, 'Severe': 3, 'Proliferate': 4}
    for label_name in label_map.keys():
        image_paths = os.listdir(os.path.join(image_dir, label_name))
        for image_name in image_paths:
            if image_name.endswith('.png'):
                image_path = os.path.join(image_dir, label_name, image_name)
                image = load_and_preprocess_image(image_path, INPUT_SIZE)
                dataset.append(image)
                labels.append(label_map[label_name])
    return np.array(dataset), np.array(labels)

# Load data
image_directory = 'dataset/'
INPUT_SIZE = 224
dataset, labels = load_data(image_directory)

# Split data into train, validation, and test sets
train_images, test_images, train_labels, test_labels = train_test_split(dataset, labels, test_size=0.2, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Define complex model architecture
inputs = tf.keras.Input(shape=(INPUT_SIZE, INPUT_SIZE, 3))

# Convolutional block 1
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2))(x)

# Convolutional block 2
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2))(x)

# Convolutional block 3
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2))(x)

# Convolutional block 4
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2))(x)

# Convolutional block 5
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2))(x)

# Flatten and fully connected layers
x = layers.Flatten()(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(5, activation='softmax')(x)

# Create model
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Model summary
model.summary()

# Define callbacks
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Increase data augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input
)

# Train the model
history = model.fit(train_datagen.flow(train_images, train_labels, batch_size=32),
                    steps_per_epoch=len(train_images) / 32,
                    epochs=10,  # Train for more epochs
                    validation_data=(val_images, val_labels),
                    callbacks=[reduce_lr, early_stop])

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

# Save the model
model.save('heart_attack_prediction_model_complex.h5')


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Function to load and preprocess images
def load_and_preprocess_image(image_path, input_size):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (input_size, input_size))
    image = image.astype('float32') / 255.0
    return image

# Load images and labels
def load_data(image_dir):
    dataset = []
    labels = []
    label_map = {'No': 0, 'Mild': 1, 'Moderate': 2, 'Severe': 3, 'Proliferate': 4}
    for label_name in label_map.keys():
        image_paths = os.listdir(os.path.join(image_dir, label_name))
        for image_name in image_paths:
            if image_name.endswith('.png'):
                image_path = os.path.join(image_dir, label_name, image_name)
                image = load_and_preprocess_image(image_path, INPUT_SIZE)
                dataset.append(image)
                labels.append(label_map[label_name])
    return np.array(dataset), np.array(labels)

# Load data
image_directory = 'dataset/'
INPUT_SIZE = 224
dataset, labels = load_data(image_directory)

# Split data into train, validation, and test sets
train_images, test_images, train_labels, test_labels = train_test_split(dataset, labels, test_size=0.2, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(INPUT_SIZE, INPUT_SIZE, 3))

# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(5, activation='softmax')(x)

# Create model
model = models.Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input
)

# Define callbacks
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(train_datagen.flow(train_images, train_labels, batch_size=32),
                    steps_per_epoch=len(train_images) / 32,
                    epochs=10,
                    validation_data=(val_images, val_labels),
                    callbacks=[reduce_lr, early_stop])

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

# Save the model
#model.save('heart_attack_prediction_model_transfer_learning.h5')


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

# Function to load and preprocess images
def load_and_preprocess_image(image_path, input_size):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (input_size, input_size))
    image = preprocess_input(image)
    return image

# Load images and labels
def load_data(image_dir):
    dataset = []
    labels = []
    label_map = {'No': 0, 'Mild': 1, 'Moderate': 2, 'Severe': 3, 'Proliferate': 4}
    for label_name in label_map.keys():
        image_paths = os.listdir(os.path.join(image_dir, label_name))
        for image_name in image_paths:
            if image_name.endswith('.png'):
                image_path = os.path.join(image_dir, label_name, image_name)
                image = load_and_preprocess_image(image_path, INPUT_SIZE)
                dataset.append(image)
                labels.append(label_map[label_name])
    return np.array(dataset), np.array(labels)

# Load data
image_directory = 'dataset/'
INPUT_SIZE = 224
dataset, labels = load_data(image_directory)

# Split data into train, validation, and test sets
train_images, test_images, train_labels, test_labels = train_test_split(dataset, labels, test_size=0.2, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Load pre-trained VGG16 model
base_model = VGG16(input_shape=(INPUT_SIZE, INPUT_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = True

# Add custom classification head
global_average_layer = layers.GlobalAveragePooling2D()
output_layer = layers.Dense(5, activation='softmax')

# Create the model
model = models.Sequential([
    base_model,
    global_average_layer,
    output_layer
])

# Compile the model
model.compile(optimizer=optimizers.Adam(lr=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_labels,
                    epochs=10,
                    batch_size=32,
                    validation_data=(val_images, val_labels),
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

# Save the model
#model.save('heart_attack_prediction_model.h5')


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

# Function to load and preprocess images
def load_and_preprocess_image(image_path, input_size):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (input_size, input_size))
    image = preprocess_input(image)
    return image

# Load images and labels
def load_data(image_dir):
    dataset = []
    labels = []
    label_map = {'No': 0, 'Mild': 1, 'Moderate': 2, 'Severe': 3, 'Proliferate': 4}
    for label_name in label_map.keys():
        image_paths = os.listdir(os.path.join(image_dir, label_name))
        for image_name in image_paths:
            if image_name.endswith('.png'):
                image_path = os.path.join(image_dir, label_name, image_name)
                image = load_and_preprocess_image(image_path, INPUT_SIZE)
                dataset.append(image)
                labels.append(label_map[label_name])
    return np.array(dataset), np.array(labels)

# Load data
image_directory = 'dataset/'
INPUT_SIZE = 224
dataset, labels = load_data(image_directory)

# Split data into train, validation, and test sets
train_images, test_images, train_labels, test_labels = train_test_split(dataset, labels, test_size=0.2, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Data augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=preprocess_input
)

# Load pre-trained VGG16 model
base_model = VGG16(input_shape=(INPUT_SIZE, INPUT_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = True

# Add custom classification head
global_average_layer = layers.GlobalAveragePooling2D()
output_layer = layers.Dense(5, activation='softmax')

# Create the model
model = models.Sequential([
    base_model,
    global_average_layer,
    output_layer
])

# Compile the model
model.compile(optimizer=optimizers.Adam(lr=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Model summary
model.summary()

# Define callbacks
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(train_datagen.flow(train_images, train_labels, batch_size=32),
                    steps_per_epoch=len(train_images) / 32,
                    epochs=50,
                    validation_data=(val_images, val_labels),
                    callbacks=[reduce_lr, early_stop])

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

# Save the model
model.save('heart_attack_prediction_model.h5')


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input

In [ ]:
# Function to load and preprocess images
def load_and_preprocess_image(image_path, input_size):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (input_size, input_size))
    image = preprocess_input(image)
    return image

In [ ]:
# Load images and labels
def load_data(image_dir):
    dataset = []
    labels = []
    label_map = {'No': 0, 'Mild': 1, 'Moderate': 2, 'Severe': 3, 'Proliferate': 4}
    for label_name in label_map.keys():
        image_paths = os.listdir(os.path.join(image_dir, label_name))
        for image_name in image_paths:
            if image_name.endswith('.png'):
                image_path = os.path.join(image_dir, label_name, image_name)
                image = load_and_preprocess_image(image_path, INPUT_SIZE)
                dataset.append(image)
                labels.append(label_map[label_name])
    return np.array(dataset), np.array(labels)

In [ ]:
# Load data
image_directory = 'dataset/'

In [ ]:
INPUT_SIZE = 224

In [ ]:
dataset, labels = load_data(image_directory)

In [ ]:
# Split data into train, validation, and test sets
train_images, test_images, train_labels, test_labels = train_test_split(dataset, labels, test_size=0.2, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

In [ ]:
# Load pre-trained DenseNet121 model (experiment with different architectures)
base_model = DenseNet121(input_shape=(INPUT_SIZE, INPUT_SIZE, 3), include_top=False, weights='imagenet')

In [ ]:
# Freeze some base model layers (adjust number of layers to unfreeze for fine-tuning)
base_model.trainable = False  # Freeze all layers by default

In [ ]:
# Optionally, select layers to unfreeze for fine-tuning (e.g., base_model.layers[-10:] are the last 10 layers)
for layer in base_model.layers[-10:]:
    layer.trainable = True

In [ ]:
# Add custom classification head
global_average_layer = layers.GlobalAveragePooling2D()
dropout = layers.Dropout(0.5)  # Add dropout for regularization
output_layer = layers.Dense(5, activation='softmax')

In [ ]:
# Create the model
model = models.Sequential([
  base_model,
  global_average_layer,
  dropout,
  output_layer
])

In [ ]:
# Compile the model
model.compile(optimizer=optimizers.Adam(lr=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(train_images, train_labels,
                    epochs=15,  # Adjust epochs as needed
                    batch_size=32,
                    validation_data=(val_images, val_labels),
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])

In [1]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

Epoch 1/15



74/74 [==============================] - 58s 735ms/step - loss: 1.3288 - accuracy: 0.5497 - val_loss: 0.9423 - val_accuracy: 0.6672
Epoch 2/15
74/74 [==============================] - 52s 708ms/step - loss: 0.9823 - accuracy: 0.6487 - val_loss: 0.8143 - val_accuracy: 0.7082
Epoch 3/15
74/74 [==============================] - 54s 738ms/step - loss: 0.8786 - accuracy: 0.6829 - val_loss: 0.7475 - val_accuracy: 0.7423
Epoch 4/15
74/74 [==============================] - 55s 747ms/step - loss: 0.8133 - accuracy: 0.7017 - val_loss: 0.7204 - val_accuracy: 0.7645
Epoch 5/15
74/74 [==============================] - 55s 750ms/step - loss: 0.7684 - accuracy: 0.7157 - val_loss: 0.7344 - val_accuracy: 0.7645
Epoch 6/15
74/74 [==============================] - 56s 762ms/step - loss: 0.7643 - accuracy: 0.7081 - val_loss: 0.7099 - val_accuracy: 0.7406
Epoch 7/15
74/74 [==============================] - 55s 744ms/step - loss: 0.7428 - accuracy: 0.7303 - val_loss: 0.6811 - val_accuracy: 0.7543
Epoch 8/15

In [2]:
# Save the model
model.save('heart_attack_prediction_model.h5')

C:\Users\Shreyas\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input

# Function to load and preprocess images
def load_and_preprocess_image(image_path, input_size):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (input_size, input_size))
    image = preprocess_input(image)
    return image

# Load images and labels
def load_data(image_dir):
    dataset = []
    labels = []
    label_map = {'No': 0, 'Mild': 1, 'Moderate': 2, 'Severe': 3, 'Proliferate': 4}
    for label_name in label_map.keys():
        image_paths = os.listdir(os.path.join(image_dir, label_name))
        for image_name in image_paths:
            if image_name.endswith('.png'):
                image_path = os.path.join(image_dir, label_name, image_name)
                image = load_and_preprocess_image(image_path, INPUT_SIZE)
                dataset.append(image)
                labels.append(label_map[label_name])
    return np.array(dataset), np.array(labels)

# Load data
image_directory = 'dataset/'
INPUT_SIZE = 224
dataset, labels = load_data(image_directory)

# Split data into train, validation, and test sets
train_images, test_images, train_labels, test_labels = train_test_split(dataset, labels, test_size=0.2, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Data augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    preprocessing_function=preprocess_input
)

# Load pre-trained DenseNet121 model
base_model = DenseNet121(input_shape=(INPUT_SIZE, INPUT_SIZE, 3), include_top=False, weights='imagenet')

# Freeze base model layers
base_model.trainable = False

# Add custom classification head
global_average_layer = layers.GlobalAveragePooling2D()
dropout = layers.Dropout(0.5)  # Add dropout for regularization
output_layer = layers.Dense(5, activation='softmax')

# Create the model
model = models.Sequential([
  base_model,
  global_average_layer,
  dropout,
  output_layer
])

# Compile the model with a lower learning rate
model.compile(optimizer=optimizers.Adam(lr=1e-5),  # Decreased learning rate
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Define a new early stopping callback with increased patience
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(train_datagen.flow(train_images, train_labels, batch_size=32),
                    steps_per_epoch=len(train_images) / 32,
                    epochs=30,
                    validation_data=(val_images, val_labels),
                    callbacks=[early_stop])

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)
# Save the model
#model.save('heart_attack_prediction_model.h5')


In [ ]:
model.save('heart_attack_prediction_model.h5')
